<img src="images/scilife_logo.png" width="400">
<img src="images/essence_logo.png" width="300">

# SciLifeLab Workshop - Hands-on Section: LangGraph "Hello World"



Welcome to this hands‑on lab session on building AI Agents with **LangGraph**! 

LangGraph is a low‑level orchestration framework for constructing stateful AI workflows using graphs. LangGraph provides several key benefits for agentic AI applications, including durable execution, support for human‑in‑the‑loop workflows, comprehensive memory (both short‑ and long‑term), built‑in debugging, and production‑ready deployment features.

## Learning Objectives
By the end of this workshop, you will:
- Understand core concepts of LangGraph (tools, nodes, edges, state, and memory).
- Create and integrate your own tools for an AI agent.
- Build a ReAct‑style agent using an LLM and custom tools.
- Implement agent memory to maintain conversational context.
- Compare custom agents with prebuilt LangGraph agents.
- Explore extension tasks such as custom graph, structured output and prompts template

## Workshop Outline 
- **Part 1**: Setup & imports
- **Part 2**: Understanding and creating tools
- **Part 3**: Defining the state
- **Part 4**: Building the agent graph
- **Part 5**: Testing the agent
- **Part 6**: Adding short‑term memory
- **Part 7**: Exploring prebuilt agents
- **Part 8 (optional)**: Extension exercises

## Instructions for Participants

**Throughout this lab, look for `TODO` comments and `...` placeholders in the code cells. These indicate where you need to add your implementation.** 

Your task is to:
1. Replace `...` placeholders with appropriate code
2. Follow the instructions in `TODO` comments 
3. Refer to the exercise descriptions and API references provided

**Tip:** Each exercise builds on the previous one, so complete them in order!

---

## Part 1 – Setup

In this first step, we'll import the necessary dependencies and load any environment variables. Make sure your API keys (e.g. OpenAI) are stored in a `.env` file in the same directory.

### Exercise 1.1 – Import dependencies

In [ ]:
# Import any packages you need here
import json
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, rdMolDescriptors
import requests
import re
import time
import pubchempy as pcp
from chembl_webresource_client.new_client import new_client
import pandas as pd
from urllib.parse import quote
from langchain.tools import tool

load_dotenv(override=True)

---

## Part 2 – Understanding and Creating Tools

In LangGraph, **tools** are Python functions that extend your agent's capabilities beyond text generation. They can call external APIs or perform computations, and are annotated with the `@tool` decorator from LangChain. 

When designing tools for agents, there are two important considerations: 
- **Input - Output**: It is similar to when designing a Python function, but the input is now handled by generated content from LLM. Output should be parsed to feed meaningful context to the agent

- **Description**: The `@tool` decorator requires docstrings from a Python function and will use it as the tool's description. These descriptions will feed the LLM system prompts, making it aware of these existing tools and their schema.

### About the task

In this exercise, we will develop three simple tools, which could be useful for the drug discovery.

**1. SMILES Resolver:** a tool to get SMILES string for compound from any identifers

**2. Get Properties:** a tool to get basic physiochemical properties of the molecule

**3. Lit Search:** a tool to perform semantic search among all publications available on PubMed

#### **Tool 1: SMILES resolver**
This tool takes any identifiers of compounds, i.e. compound name, chembl id, pubchem cid, cas number, etc. It returns a dictionary containing original identifiers and queried SMILES string

In [ ]:
# Initialize connection to external API
CACTUS_BASE = "https://cactus.nci.nih.gov/chemical/structure"
molecule = new_client.molecule

# Helper functions
def _is_chembl_id(value: str) -> bool:
    return bool(re.fullmatch(r"CHEMBL\d+", value.upper()))

def _looks_like_inchikey(value: str) -> bool:
    return bool(re.fullmatch(r"[A-Z]{14}-[A-Z]{10}-[A-Z]", value))


# Main tool
@tool
def resolve_smiles(
    identifier: str,
    pause_s: float = 0.0,
    timeout_s: float = 15.0,
) -> dict | None:
    """
    Resolve any identifier types to canonical SMILES.
    
    Args:
        identifier: any identifiers of compounds, i.e. name, CID, CHEMBL, CAS, etc.
    
    Returns: 
        dict: original identifiers and corresponding SMILES string
    """
    ident = identifier.strip()
    if not ident:
        return None
    
    ident_lower = ident.lower()
    ident_upper = ident.upper()
    
    # Try ChEMBL
    if _is_chembl_id(ident_upper):
        try:
            mol = molecule.get(ident_upper)
            smiles = mol.get("molecule_structures", {}).get("canonical_smiles")
            if smiles:
                if pause_s:
                    time.sleep(pause_s)
                return {'identifier': identifier, 'SMILES': smiles}
        except Exception:
            pass
    
    
    # Try CACTUS
    try:
        url = f"{CACTUS_BASE}/{quote(ident)}/smiles"
        response = requests.get(url, timeout=timeout_s, headers={"User-Agent": "smiles-resolver/1.0"})
        if response.ok:
            text = response.text.strip()
            # Explicitly check for non-empty and valid content
            if (text  and len(text) > 0 ):
                if pause_s:
                    time.sleep(pause_s)
                return {'identifier': identifier, 'SMILES': text}
            # If empty or invalid, fall through to PubChem
    except requests.RequestException:
        pass
    
    # Try PubChem
    try:
        if ident.isdigit():
            compound = pcp.Compound.from_cid(int(ident))
            smiles = getattr(compound, "canonical_smiles", None)
            if smiles:
                if pause_s:
                    time.sleep(pause_s)
                return {'identifier': identifier, 'SMILES': smiles}
        
        if _looks_like_inchikey(ident_upper):
            compounds = pcp.get_compounds(ident_upper, namespace="inchikey")
        else:
            compounds = pcp.get_compounds(ident_lower, namespace="name")
        
        if compounds:
            smiles = getattr(compounds[0], "canonical_smiles", None)
            if smiles:
                if pause_s:
                    time.sleep(pause_s)
                return {'identifier': identifier, 'SMILES': smiles}
    except Exception:
        pass
    
    return None

#### **Tool 2: Get Properties**
This tool takes SMILES string as input and output a dictionary containing SMILES string and its physiochemical properties, i.e. MW, logP, HBA, HBA, TPSA, etc

In [ ]:
@tool 
def get_properties(smiles: str):
    # TODO: Write description for this function
    """
    What does it do?

    What are the args?

    What are the output (return)?
    """
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return {"error": "Invalid SMILES string"}
    
    # TODO: Gat all desired properties
    # You can get more properties if you want
    properties = {
        "Molecular_Weight": Descriptors..., # get Molecular Weight
        "LogP": Descriptors..., # get Molecular logP
        "HBD": Descriptors...,  # get number of hydrogen bond donors
        "HBA": Descriptors...,  # get number of hydrogen bond acceptors
        "TPSA": Descriptors...,  # get TPSA
        "Rotatable_Bonds": Descriptors..., # get number of rotatable bonds
        "Aromatic_Rings": Descriptors..., # get number of aromatic rings
        "Heavy_Atoms": Descriptors..., # get number of heavy atoms
        "Formal_Charge": Chem.GetFormalCharge(mol),
    }
    
    return {
        'SMILES': smiles,
        'Properties': properties
    }


#### **Tool 3: Lit Search**
This tool processes a natural language query and uses semantic search to retrieve the most relevant passages from all PubMed publications.

In [ ]:
from langchain.tools import tool
from utils.litsense.litsense import LitSense_API

@tool
def lit_search(query: str, limit: int = 20, rerank: bool =True) -> str:
    """Retrieve information from PubMed using a semantic search via the LitSense API.\n\n
    
    Args:
        query: The research question or topic to search for in PubMed literature.
        limit: Maximum number of results to return (default is 5).
        rerank: Whether to rerank the relevance of retrieved paragraphs.
    
    Returns:
        str: all the paragraphs + corresponding PMID number.
    
    """

    try:
        engine = LitSense_API()
        results = engine.retrieve(query, limit=limit, rerank=rerank)
        if not results:
            return f"No relevant literature found for '{query}'. Please try a different or broader search query."
        result_str = ""
        for i, result in enumerate(results):
            result_str += (
                f"\n--- Passage #{i+1} ---\n"
                f"PMID: {result.pmid}\n"
                f"Content: {result.text}\n"
            )
        return result_str
    except Exception as e:
        return f"Error retrieving literature for '{query}': {str(e)}. Please try a different search query."

### Exercise 2.4 – Create a Tools List

Collect all of your tool functions into a single list called `TOOLS`. This list will be passed to the LLM so that it is aware of what tools are available.

In [ ]:
# TODO: Put all your tools function into TOOLS list
TOOLS = [...]

---

## Part 3 – Understanding LangGraph State

For agents, the state typically contains a list of messages which grows over the conversation. 

When creating a state schema, you use `TypedDict` to describe the keys and `Annotated` with updating functions to specify how values should be updated. 

The `add_messages` function appends new messages rather than overwriting them.


### Exercise 3.1 – Define Chat State

Define a `ChatState` class (subclassing `TypedDict`) with a single key `messages`. Use the `add_messages` updating function so that new messages are appended to the state.

<img src="images/ChatState.png" width="800">

In [ ]:
from typing_extensions import TypedDict
from typing import Annotated, List, Dict
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    """The state schema for our LangGraph. It contains only a list of messages.

    Messages are appended via the `add_messages` to preserve the full conversation history."""
    messages: Annotated[List[Dict], add_messages]

---

## Part 4 – Building the Agent Graph

This is the graph we are going to create: 

<img src="images/react_agent.png" width="300">

### Exercise 4.1 – Initialize Components

1. Initialise a `StateGraph` instance using your `ChatState`.
2. Initialise a chat model with `ChatOpenAI(model='gpt-4o', temperature=0)`.
3. Bind your tools to the LLM using `bind_tools()`.
4. Create a `ToolNode` from your tools list.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI

# Create graph builder using ChatState
graph_builder = StateGraph(ChatState)

# TODO: Initialise the chat model 
llm = ChatOpenAI(model='...', temperature=0)

# TODO: Bind the tools to the LLM so that it knows their schemas and how to construct tool calls in JSON
llm_with_tools = llm.bind_tools(..., parallel_tool_calls=False)

# TODO: Create a ToolNode using our tools list
tool_node = ToolNode(tools=...)

### Exercise 4.2 – Define the Chatbot Node

Define a chatbot function that takes the `state` as input and returns a dictionary with a single key `messages`.

When invoke the LLM, we provide it with all the `messages` in `state`.

In [ ]:
from typing import Dict, List

def chatbot(state: ChatState) -> Dict[str, List]:
    """The main chatbot node. It invokes the LLM with the current messages."""
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

### Exercise 4.3 – Define the Routing Function

Create a function `route_tools` that decides whether to call tools or terminate. 

- Step 1: Extract the last message from `state`.
- Step 2: If the last message contains tool calls (`tool_calls` attribute), return `'tools'`; otherwise return `'END'`.

In [ ]:
# TODO: complete the output of route function
def route_tools(state: ChatState) -> str:
    messages = state.get('messages', [])
    ai_message = messages[-1] if messages else None
    if ai_message and getattr(ai_message, 'tool_calls', []):
        return ...
    return ...

### Exercise 4.4 – Build the Complete Graph

Assemble the agent graph by adding nodes and edges, then compile the graph into a runnable agent. Use `add_node()` for your chatbot and tool nodes, `add_conditional_edges()` for routing logic, and `add_edge()` to connect nodes.


<img src="images/graph_create_1.png" width="800">

<img src="images/graph_create_2.png" width="800">

In [ ]:
graph_builder = StateGraph(ChatState)

# TODO: Add the chatbot node to the graph
graph_builder.add_node('...', ...)

# TODO: Add the tool node to the graph
graph_builder.add_node('...', ...)

# TODO: Add conditional edges from chatbot using the routing function
graph_builder.add_conditional_edges('...', route_tools, {'...': '...', '...': ...})

# TODO: Add edge from tools back to chatbot
graph_builder.add_edge('...', '...')

# TODO: Add edge from START to chatbot
graph_builder.add_edge(..., '...')

# Compile the graph into an agent
agent = graph_builder.compile()

### Exercise 4.5 – Visualize Your Agent

Use the graph's `draw_mermaid_png()` method to visualize the structure of your agent. This step is optional but helps you understand how nodes and edges connect.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception:
    print('Graph visualization not available in this environment.')

---

## Part 5 – Testing Your Agent

Now that your agent graph is built, it's time to interact with it. Create a simple chat loop that greets the user, processes input until they type 'quit', and streams responses using `agent.stream()`.

### Exercise 5.1 – Create a Basic Chat Loop

Write an interactive loop that:
1. Greets the user and explains the agent's capabilities.
2. Reads user input in a loop and exits on `'quit'`, `'exit'` or `'q'`.
3. Creates the initial `state` with the user's message and streams the agent's responses via `agent.stream()`.
4. Uses the `pretty_print()` method on messages to display nicely formatted output.


Try out this chat loop with these three prompts: 

- **Prompt 1**: ” You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. Here is the compound ID: Erlotinib. Is it orally drug-like?”


- **Prompt 2**: " You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. Which is more lipophilic: Imatinib or Dasatinib?”


- **Prompt 3**:  " You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. Check betulinic acid. Are there any structural red flags for oral bioavailability?”

- **Prompt 4**:  " You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. What are known EGFR inhibitors in the literature, and are they drug-like?”

- **Prompt 5**:  " You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. What are the common physicochemical characteristics of successful JAK inhibitors?"




In [ ]:
print('Welcome to the LangGraph ReAct demo with memory! Ask me a question.')
print('I can perform simple math, look up drug information, and search PubMed via LitSense.')
print('====================================================================')

while True:
    try:
        # Get input prompt from user
        user_input = input('User: ')
    except EOFError:
        break
    
    # If input contains one of three keywords: quit, exit, and q; exit the streaming process.
    if not user_input or user_input.lower() in {'quit', 'exit', 'q'}:
        print('Goodbye!')
        break
    
    # Initialize graph state with user's input
    state = {
        'messages': [
            {'role': 'user', 'content': user_input}
        ]
    }

    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    # Stream the answer from agent
    for event in agent.stream(state):
        for value in event.values():
            # The last message in the state is the AI response
            msg = value["messages"][-1]
            # Use the built-in pretty_print method for better formatting
            msg.pretty_print()

---

## Part 6: Adding Short‑term Memory

To demonstrate the lack of memory in the agent, please **go back to Part 5**, then perform these two prompts: 

- **Prompt 1:** Here is the compound ID: Erlotinib. Is it orally drug-like?

- **Prompt 2:** Which is more lipophilic: Imatinib or the previous drug?

You will see when invoking **prompt 2**, the agent get confuse since it doesn't know the previous drug is Erlotinib. This is because the GraphState is only maintained from the START node to the END node. After termination, all contexts from the previous run are lost. Therefore, we need a component called Agent Memory to maintain context across runs.

Memory allows your agent to remember previous parts of the conversation. LangGraph uses "checkpointers" to maintain state across interactions.

### Exercise 6.1 – Add Memory to Your Agent

Rebuild your agent with memory by creating a new `StateGraph` that uses the same `ChatState`. 

Initialize `checkpointer` with `InMemorySaver` to persist state across turns.

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

class ChatState(TypedDict):
    messages: Annotated[List[Dict], add_messages]

graph_builder = StateGraph(ChatState)

llm = ChatOpenAI(model='gpt-4o', temperature=0)
llm_with_tools = llm.bind_tools(TOOLS, parallel_tool_calls=False)

def chatbot(state: ChatState) -> Dict[str, List]:
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

tool_node = ToolNode(tools=TOOLS)

graph_builder.add_node('chatbot', chatbot)
graph_builder.add_node('tools', tool_node)
graph_builder.add_conditional_edges('chatbot', route_tools, {'tools': 'tools', 'END': END})
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')

checkpointer = InMemorySaver()
agent = graph_builder.compile(checkpointer=checkpointer)

### Exercise 6.2 – Create a Memory‑Enabled Chat Loop

Write a chat loop similar to Part 5, but supply a `config` dictionary to `agent.stream()`. In the config, there are two important parameters, including `thread_id` and `recursion_limit`.

<img src="images/short_term_mem.png" width="800">

In [ ]:
# TODO: Create config for memory
config = {
    'configurable': {
        'thread_id': '...',
    },
    'recursion_limit': ...
}

In [ ]:
print('Welcome to the LangGraph ReAct demo with memory! Ask me a question.')
print('I can perform simple math, look up drug information, and search PubMed via LitSense.')
print('====================================================================')

while True:
    try:
        # Get input prompt from user
        user_input = input('User: ')
    except EOFError:
        break
    
    # If input contains one of three keywords: quit, exit, and q; exit the streaming process.
    if not user_input or user_input.lower() in {'quit', 'exit', 'q'}:
        print('Goodbye!')
        break

    # Initialize graph state with user's input
    state = {
        'messages': [
            {'role': 'user', 'content': user_input}
        ]
    }


    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    
    # TODO: Supply config for the agent stream process
    for event in agent.stream(state, config=...):
        for value in event.values():
            # The last message in the state is the AI response
            msg = value["messages"][-1]
            # Use the built-in pretty_print method for better formatting
            msg.pretty_print()

---

## Part 7 – Prebuilt Agents

LangGraph provides prebuilt agents that implement common architectures such as the ReAct pattern. These agents are quick to set up and let you focus on your tools and prompts rather than on graph wiring.

### Exercise 7.1 – Create a Prebuilt Agent

Use `create_agent` to instantiate a prebuilt ReAct agent. Provide your LLM, tools list, and a custom system prompt that instructs the agent how to behave.

#### References: [LangGraph Prebuilt Tutorial](https://langchain-ai.github.io/langgraph/agents/agents)

In [ ]:
# Create config for memory
config = {
    'configurable': {
        'thread_id': 'example_conversation',
    },
    'recursion_limit': 25
}

In [ ]:
from langchain.agents import create_agent

llm = ChatOpenAI(model='gpt-4o', temperature=0)

checkpointer = InMemorySaver()

agent = create_agent(
    model = llm,
    tools = TOOLS,
    system_prompt="You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data.",
    checkpointer = checkpointer
)

In [ ]:
print('Welcome to the LangGraph ReAct demo with memory! Ask me a question.')
print('I can perform simple math, look up drug information, and search PubMed via LitSense.')
print('====================================================================')

while True:
    try:
        # Get input prompt from user
        user_input = input('User: ')
    except EOFError:
        break
    
    # If input contains one of three keywords: quit, exit, and q; exit the streaming process.
    if not user_input or user_input.lower() in {'quit', 'exit', 'q'}:
        print('Goodbye!')
        break

    # Initialize graph state with user's input
    state = {
        'messages': [
            {'role': 'user', 'content': user_input}
        ]
    }

    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    
    # Supply config for the agent stream process
    for event in agent.stream(state, config=config):
        for value in event.values():
            # The last message in the state is the AI response
            msg = value["messages"][-1]
            # Use the built-in pretty_print method for better formatting
            msg.pretty_print()

---

## Part 8 – Extension Exercises


### Exercise 8.1 – Create a system prompt using a prompt template

In LangGraph, you can provide a **system** or **instruction** message that sets the behaviour of the assistant before any user input is processed. LangGraph exposes this functionality via its `prompts` module. There you will find a few classes: `PromptTemplate` for formatting a single string, `ChatPromptTemplate` for building multi-message prompts, and `MessagesPlaceholder` for inserting a slot where conversation history will later be filled.

In this exercise you will define a `ChatPromptTemplate` that begins with a system instruction. The system message should describe the assistant’s role (for example, instructing it to behave like a drug discovery researcher) and will be prepended to the user’s request before being passed to the agent.

#### API References: [Prompt Template](https://python.langchain.com/docs/concepts/prompt_templates/)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data."),
    ("user", "{request}")
])

prompt_values = prompt_template.invoke({"request": "What is the mechanism of action of remdesivir and what bioactivity data is available for this compound?"})

`prompt_values` is a dictionary, which can be invoke directly to the agent. Try out by simple invoke the agent with the `prompt_values`

In [ ]:
msgs = agent.invoke(prompt_values, config=config)
for msg in msgs['messages']:
    msg.pretty_print()

### Exercise 8.2 – Integrate the system prompt with a streaming loop

Building on the previous exercise, implement a multi-turn chat loop. Use a `while` loop to repeatedly accept user input, stream the agent’s response, and append both user and assistant messages to the conversation state.

To include your system instruction in every turn, apply the `ChatPromptTemplate` from exercise 8.1 when constructing the input for the agent. 

When using the streaming API you must supply a `state` dictionary that includes a `'messages'` key. The value of this key should be a list of message dictionaries (each with `role` and `content` fields) representing the conversation history. 

As you loop, update this list to maintain the context across turns.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

config = {
    "configurable": {
        "thread_id": "example_conversation"
    },
    "recursion_limit": 25
}


prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data."),
    ("human", "{request}"),
])

print("Welcome to the LangGraph ReAct demo with memory! Ask me a question.")
print("I can perform simple math, look up drug information, and search PubMed via LitSense.")
print("====================================================================")

while True:
    try:
        user_input = input("User: ")
    except EOFError:
        break

    if not user_input or user_input.lower() in {"quit", "exit", "q"}:
        print("Goodbye!")
        break

    # Build the state as a dict with a messages list
    prompt_values = prompt_template.invoke({"request": user_input})
    state = {"messages": prompt_values.messages}

    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)

    # Stream agent events
    for event in agent.stream(state, config=config):
        for value in event.values():
            msg = value["messages"][-1]
            msg.pretty_print()

### Exercise 9.1 - Structure Output Parser

Structured output parsers let you control the format of the language model’s responses. Instead of returning free‑form text, the model follows a schema (such as a Pydantic model) so that other components can reliably consume its output.

- **Why?** Downstream software often expects data in a specific JSON schema (e.g. key–value pairs).
- **How?** Define a Pydantic `BaseModel` to describe the fields you need, then use `with_structured_output(...)` to attach this schema to your LLM.

You can read more in the [LangGraph structured output guide](https://langchain-ai.github.io/langgraph/how-tos/react-agent-structured-output/#define-model-tools-and-graph-state).

The diagram below shows the high‑level structure of the system we’re about to build. One LLM is responsible for deciding when to call tools, and a second LLM is tasked with formatting the final response according to your schema.

<img src="images/react_structured_output.png" width="300">


First, we'll set up two language models:

1. **Tool‑calling LLM**: This model can invoke the tools you defined earlier (calculator, literature search, etc.).
2. **Structured output LLM**: This model will use a Pydantic schema to return the final answer as JSON with fields for drug names, justifications, and sources.

When we later ask a question such as *"Use LitSearch literature, give me two antiviral drugs that are potentially to treat COVID‑19"*, the agent will search PubMed via LitSense, find candidate antivirals, and then format its answer using this schema.



In [ ]:
# Step 1: create the tool-calling LLM
llm = ChatOpenAI(model='gpt-5', temperature=0)
llm_with_tools = llm.bind_tools(TOOLS)

# Step 2: define a Pydantic schema describing the structured output we want
from pydantic import BaseModel, Field

class DrugsInfo(BaseModel):
    drugs: str = Field(description='a list of drug names')
    justifications: str = Field(description='a list of justification for each drugs')
    sources: str = Field(description='a list of citation for each drugs')

# Step 3: create the structured output LLM
llm = ChatOpenAI(model='gpt-5', temperature=0)
llm_with_structured_outputs = llm.with_structured_output(DrugsInfo)


In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from typing import Dict, List
from langchain_core.messages import HumanMessage, AIMessage

# Define the chatbot node: this function invokes the tool‑calling LLM and appends the AI message to state
def chatbot(state: ChatState) -> Dict[str, List]:
    """The main chatbot node. It invokes the LLM with the current messages."""
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

# Define the respond node: this function uses the structured‑output LLM to format the final answer
def respond(state: ChatState):
    # Extract only the content of each message for the structured LLM
    response = llm_with_structured_outputs.invoke(
        [HumanMessage(content=state["messages"][i].content) for i in range(len(state["messages"]))]
    )
    # Wrap the structured JSON as an AIMessage and return it
    return {"messages": [AIMessage(content=response.model_dump_json())]}

# Define a simple routing function to decide whether to keep calling tools or format a final response
def route_tools(state: ChatState):
    messages = state["messages"]
    last_message = messages[-1]
    # If the last message does not contain a tool call, we are ready to format a response
    if not last_message.tool_calls:
        return "respond"
    return "continue"


In [ ]:
# Build the state graph
workflow = StateGraph(ChatState)

# Add the nodes: chatbot (for reasoning/tool calls), respond (for final formatting), and the existing tool node
workflow.add_node("chatbot", chatbot)
workflow.add_node("respond", respond)
workflow.add_node("tools", tool_node)

# Set the entry point: we start by invoking the chatbot node
workflow.add_edge(START, "chatbot")

# Add conditional edges: route to either tools or respond depending on should_continue
workflow.add_conditional_edges(
    "chatbot",
    route_tools,
    {
        "continue": "tools",
        "respond": "respond",
    },
)

# After calling tools, loop back to the chatbot; after responding, end the workflow
workflow.add_edge("tools", "chatbot")
workflow.add_edge("respond", END)

# Compile the workflow into an executable agent
agent = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception:
    print('Graph visualization not available in this environment.')

In [ ]:
# Ask the agent a question and pretty‑print the structured response
answer = agent.invoke({'messages': 'Use LitSearch literatures, give me 2 antiviral drugs that are potential to treat COVID-19'})
answer['messages'][-1].pretty_print()

---

## Key Takeaways

- **Nodes** are functions that operate on state.
- **Edges** define the flow between nodes.
- **State** carries data through the agent. Use reducer functions such as `add_messages` to control how the state is updated.
- **Tools** extend agent capabilities and are annotated with `@tool`.
- **Memory** allows agents to maintain context across invocations.
- **Prebuilt agents** provide quick solutions for common patterns but offer less control than a custom graph.

---
## Resources

- **LangChain Documentation** – https://docs.langchain.com/oss/python/langchain/overview
- **LangGraph Documentation** – https://docs.langchain.com/oss/python/langgraph/overview
- **ChEMBL Web Services** – https://chembl.gitbook.io/chembl-interface-documentation/web-services
- **OpenAI API** – https://platform.openai.com/docs/

These references can help you explore LangGraph and related libraries beyond the scope of this workshop.

---

**🎉 Congratulations! You've built your first AI agent with LangGraph!**

Feel free to modify and extend your agent. You can experiment with new tools, different LLMs, and additional nodes to create even more capable and personalised agents.